In [13]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -2.1667,
	"longitude": -79.9,
	"current": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "is_day", "precipitation", "rain", "showers", "snowfall", "weather_code", "cloud_cover", "pressure_msl", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()
current_apparent_temperature = current.Variables(2).Value()
current_is_day = current.Variables(3).Value()
current_precipitation = current.Variables(4).Value()
current_rain = current.Variables(5).Value()
current_showers = current.Variables(6).Value()
current_snowfall = current.Variables(7).Value()
current_weather_code = current.Variables(8).Value()
current_cloud_cover = current.Variables(9).Value()
current_pressure_msl = current.Variables(10).Value()
current_surface_pressure = current.Variables(11).Value()
current_wind_speed_10m = current.Variables(12).Value()
current_wind_direction_10m = current.Variables(13).Value()
current_wind_gusts_10m = current.Variables(14).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")
print(f"Current relative_humidity_2m {current_relative_humidity_2m}")
print(f"Current apparent_temperature {current_apparent_temperature}")
print(f"Current is_day {current_is_day}")
print(f"Current precipitation {current_precipitation}")
print(f"Current rain {current_rain}")
print(f"Current showers {current_showers}")
print(f"Current snowfall {current_snowfall}")
print(f"Current weather_code {current_weather_code}")
print(f"Current cloud_cover {current_cloud_cover}")
print(f"Current pressure_msl {current_pressure_msl}")
print(f"Current surface_pressure {current_surface_pressure}")
print(f"Current wind_speed_10m {current_wind_speed_10m}")
print(f"Current wind_direction_10m {current_wind_direction_10m}")
print(f"Current wind_gusts_10m {current_wind_gusts_10m}")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(8).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(9).ValuesAsNumpy()
daily_uv_index_clear_sky_max = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_showers_sum = daily.Variables(13).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(14).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(15).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(17).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(18).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(19).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(20).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(21).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["uv_index_clear_sky_max"] = daily_uv_index_clear_sky_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)


Coordinates -2.125°N -79.875°E
Elevation 8.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1733444100
Current temperature_2m 27.399999618530273
Current relative_humidity_2m 65.0
Current apparent_temperature 29.26337432861328
Current is_day 0.0
Current precipitation 0.0
Current rain 0.0
Current showers 0.0
Current snowfall 0.0
Current weather_code 3.0
Current cloud_cover 93.0
Current pressure_msl 1009.5999755859375
Current surface_pressure 1008.6824951171875
Current wind_speed_10m 14.512063980102539
Current wind_direction_10m 226.00498962402344
Current wind_gusts_10m 29.51999855041504
                       date  weather_code  temperature_2m_max  \
0 2024-12-06 00:00:00+00:00          61.0           27.706501   
1 2024-12-07 00:00:00+00:00          61.0           29.206501   
2 2024-12-08 00:00:00+00:00           3.0           30.206501   
3 2024-12-09 00:00:00+00:00           3.0           30.456501   
4 2024-12-10 00:00:00+00:00           3.0           31.7065

In [14]:
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [15]:
daily_dataframe

,date,weather_code,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,daylight_duration,sunshine_duration,uv_index_max,uv_index_clear_sky_max,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2024-12-06 00:00:00+00:00,61.0,27.706501,23.006500,29.551395,25.376415,0,0,44057.613281,208.261414,8.90,8.95,0.4,0.2,0.2,0.0,4.0,33.0,14.773164,29.519999,277.636444,6.620000,2.085462
1,2024-12-07 00:00:00+00:00,61.0,29.206501,22.456501,32.250301,25.923449,0,0,44060.304688,12069.456055,6.75,8.90,2.2,2.1,0.0,0.0,10.0,18.0,7.636753,15.119999,198.946594,10.500000,2.417361
2,2024-12-08 00:00:00+00:00,3.0,30.206501,22.756500,33.061180,25.730936,0,0,44062.832031,29760.000000,8.85,8.90,0.0,0.0,0.0,0.0,0.0,10.0,10.495713,19.440001,214.149231,13.500000,3.342853
3,2024-12-09 00:00:00+00:00,3.0,30.456501,23.106501,32.820526,25.477852,0,0,44065.191406,31125.035156,8.90,8.90,0.0,0.0,0.0,0.0,0.0,3.0,12.261157,23.400000,220.782196,13.280000,3.478357
4,2024-12-10 00:00:00+00:00,3.0,31.706501,23.556499,35.066914,26.326092,0,0,44067.375000,33355.312500,8.85,8.90,0.0,0.0,0.0,0.0,0.0,0.0,11.923557,21.959999,216.869934,16.559999,4.131252
5,2024-12-11 00:00:00+00:00,3.0,32.456501,23.206501,35.648071,25.643423,0,0,44069.375000,29334.078125,8.45,8.40,0.0,0.0,0.0,0.0,0.0,0.0,14.336861,27.000000,220.780075,16.530001,4.047151
6,2024-12-12 00:00:00+00:00,3.0,32.706501,23.656500,35.669487,26.498400,0,0,44071.195312,36760.828125,7.05,8.35,0.0,0.0,0.0,0.0,0.0,0.0,10.086427,21.959999,213.736801,15.980000,3.906188


In [16]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -2.1667,
	"longitude": -79.9,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(8).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(9).ValuesAsNumpy()
daily_uv_index_clear_sky_max = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_showers_sum = daily.Variables(13).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(14).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(15).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(17).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(18).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(19).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(20).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(21).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["uv_index_clear_sky_max"] = daily_uv_index_clear_sky_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates -2.125°N -79.875°E
Elevation 8.0 m asl
Timezone b'America/Guayaquil' b'-05'
Timezone difference to GMT+0 -18000 s
                       date  weather_code  temperature_2m_max  \
0 2024-12-05 05:00:00+00:00           3.0           33.206501   
1 2024-12-06 05:00:00+00:00          61.0           26.706501   
2 2024-12-07 05:00:00+00:00          61.0           29.206501   
3 2024-12-08 05:00:00+00:00           3.0           30.206501   
4 2024-12-09 05:00:00+00:00           3.0           30.456501   
5 2024-12-10 05:00:00+00:00           3.0           31.706501   
6 2024-12-11 05:00:00+00:00           3.0           32.456501   

   temperature_2m_min  apparent_temperature_max  apparent_temperature_min  \
0           23.256500                 37.864140                 25.791853   
1           23.006500                 29.342323                 25.376415   
2           22.456501                 32.250301                 25.923449   
3           22.756500                 33.0611

In [17]:
daily_dataframe

,date,weather_code,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,daylight_duration,sunshine_duration,uv_index_max,uv_index_clear_sky_max,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2024-12-05 05:00:00+00:00,3.0,33.206501,23.256500,37.864140,25.791853,0,0,44055.367188,37362.300781,8.95,8.95,0.3,0.0,0.3,0.0,3.0,20.0,17.727943,33.480000,218.915268,21.389999,5.208746
1,2024-12-06 05:00:00+00:00,61.0,26.706501,23.006500,29.342323,25.376415,0,0,44058.187500,208.261414,8.90,8.95,0.5,0.4,0.0,0.0,5.0,33.0,11.457958,24.480000,350.537720,6.620000,1.798939
2,2024-12-07 05:00:00+00:00,61.0,29.206501,22.456501,32.250301,25.923449,0,0,44060.843750,12069.456055,6.75,8.90,1.9,1.9,0.0,0.0,7.0,18.0,9.931042,18.359999,205.463287,10.500000,2.571988
3,2024-12-08 05:00:00+00:00,3.0,30.206501,22.756500,33.061180,25.730936,0,0,44063.335938,29760.000000,8.85,8.90,0.0,0.0,0.0,0.0,0.0,10.0,12.261157,23.400000,215.852997,13.500000,3.413857
4,2024-12-09 05:00:00+00:00,3.0,30.456501,23.106501,32.820526,25.477852,0,0,44065.656250,31125.035156,8.90,8.90,0.0,0.0,0.0,0.0,0.0,3.0,10.514218,19.799999,219.245407,13.300000,3.459378
5,2024-12-10 05:00:00+00:00,3.0,31.706501,23.556499,35.066914,26.170549,0,0,44067.804688,33355.312500,8.85,8.90,0.0,0.0,0.0,0.0,0.0,0.0,14.336861,27.000000,216.641220,16.559999,4.186162
6,2024-12-11 05:00:00+00:00,3.0,32.456501,23.206501,35.648071,25.643423,0,0,44069.769531,29334.078125,8.45,8.40,0.0,0.0,0.0,0.0,0.0,0.0,12.727921,25.199999,219.101471,16.530001,3.936365


In [22]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility", "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m"],
	"timezone": "auto",
	"past_days": 1,
	"past_minutely_15": 1,
	"forecast_days": 1,
	"forecast_minutely_15": 4
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_rain = hourly.Variables(6).ValuesAsNumpy()
hourly_showers = hourly.Variables(7).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(8).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(9).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(10).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(11).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(14).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(15).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(16).ValuesAsNumpy()
hourly_visibility = hourly.Variables(17).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(18).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(19).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(20).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(21).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["visibility"] = hourly_visibility
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                        date  temperature_2m  relative_humidity_2m  \
0  2024-12-04 23:00:00+00:00          4.1455                  94.0   
1  2024-12-05 00:00:00+00:00          4.2455                  93.0   
2  2024-12-05 01:00:00+00:00          4.2955                  93.0   
3  2024-12-05 02:00:00+00:00          4.2955                  94.0   
4  2024-12-05 03:00:00+00:00          4.2455                  92.0   
5  2024-12-05 04:00:00+00:00          4.1955                  93.0   
6  2024-12-05 05:00:00+00:00          4.1955                  93.0   
7  2024-12-05 06:00:00+00:00          3.9955                  94.0   
8  2024-12-05 07:00:00+00:00          3.7455                  93.0   
9  2024-12-05 08:00:00+00:00          3.6455                  91.0   
10 2024-12-05 09:00:00+00:00          3.5455                  89.0   
11 2024-12-05 10:

In [21]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m
0,2024-12-05 23:00:00+00:00,2.8455,68.0,-2.479957,-2.146340,18.0,0.0,0.0,0.0,0.0,0.0,3.0,1009.900024,1005.163208,100.0,93.0,96.0,100.0,52160.0,0.0,0.029828,0.240532,17.238699
1,2024-12-06 00:00:00+00:00,3.0455,69.0,-2.090831,-2.151215,40.0,0.0,0.0,0.0,0.0,0.0,3.0,1008.400024,1003.673706,100.0,88.0,100.0,100.0,51800.0,0.0,0.031413,0.236342,18.998316
2,2024-12-06 01:00:00+00:00,3.0455,71.0,-1.702946,-2.005261,63.0,0.2,0.1,0.0,0.0,0.0,61.0,1007.200012,1002.479309,100.0,100.0,100.0,100.0,49140.0,0.0,0.027893,0.221097,18.359999
3,2024-12-06 02:00:00+00:00,3.0455,73.0,-1.324636,-2.003381,68.0,0.1,0.1,0.0,0.0,0.0,61.0,1005.000000,1000.289612,100.0,100.0,100.0,100.0,23000.0,0.0,0.025713,0.205851,18.709612
4,2024-12-06 03:00:00+00:00,2.8455,75.0,-1.148965,-2.244299,80.0,1.2,1.2,0.0,0.0,0.0,61.0,1003.099976,998.395081,100.0,100.0,100.0,100.0,22260.0,0.0,0.023341,0.187923,19.161337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2024-12-12 18:00:00+00:00,1.2195,89.0,-0.392896,-1.398341,7.0,0.0,0.0,0.0,0.0,0.0,3.0,1031.500000,1026.633179,95.0,83.0,75.0,0.0,24140.0,0.0,0.000000,0.073631,2.968636
164,2024-12-12 19:00:00+00:00,1.2695,90.0,-0.189818,-1.357191,6.0,0.0,0.0,0.0,0.0,0.0,3.0,1031.500000,1026.634155,97.0,89.0,69.0,0.0,24140.0,0.0,0.000000,0.067178,3.240000
165,2024-12-12 20:00:00+00:00,1.3695,91.0,0.061345,-1.333090,6.0,0.0,0.0,0.0,0.0,0.0,3.0,1031.500000,1026.635742,98.0,94.0,63.0,0.0,24140.0,0.0,0.000000,0.060896,4.024922
166,2024-12-12 21:00:00+00:00,1.4695,92.0,0.311276,-1.271198,6.0,0.0,0.0,0.0,0.0,0.0,3.0,1031.500000,1026.637695,100.0,100.0,57.0,0.0,24140.0,0.0,0.000000,0.054520,4.553679


In [25]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility", "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m"],
	"timezone": "auto",
	"past_days": 1,
	"past_minutely_15": 1,
	"forecast_days": 1,
	"forecast_minutely_15": 4
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_rain = hourly.Variables(6).ValuesAsNumpy()
hourly_showers = hourly.Variables(7).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(8).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(9).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(10).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(11).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(14).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(15).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(16).ValuesAsNumpy()
hourly_visibility = hourly.Variables(17).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(18).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(19).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(20).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(21).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["visibility"] = hourly_visibility
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s
                        date  temperature_2m  relative_humidity_2m  \
0  2024-12-04 23:00:00+00:00          4.1455                  94.0   
1  2024-12-05 00:00:00+00:00          4.2455                  93.0   
2  2024-12-05 01:00:00+00:00          4.2955                  93.0   
3  2024-12-05 02:00:00+00:00          4.2955                  94.0   
4  2024-12-05 03:00:00+00:00          4.2455                  92.0   
5  2024-12-05 04:00:00+00:00          4.1955                  93.0   
6  2024-12-05 05:00:00+00:00          4.1955                  93.0   
7  2024-12-05 06:00:00+00:00          3.9955                  94.0   
8  2024-12-05 07:00:00+00:00          3.7455                  93.0   
9  2024-12-05 08:00:00+00:00          3.6455                  91.0   
10 2024-12-05 09:00:00+00:00          3.5455                  89.0   
11 2024-12-05 10:

AttributeError: 'NoneType' object has no attribute 'Time'

In [26]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m
0,2024-12-04 23:00:00+00:00,4.1455,94.0,3.266053,1.517526,63.0,0.2,0.2,0.0,0.0,0.0,61.0,1024.199951,1019.418457,100.0,96.0,100.0,0.0,4040.0,0.0,0.000000,0.049433,7.289444
1,2024-12-05 00:00:00+00:00,4.2455,93.0,3.213841,1.563219,55.0,0.2,0.2,0.0,0.0,0.0,61.0,1025.099976,1020.315857,100.0,100.0,100.0,0.0,5180.0,0.0,0.000000,0.058077,7.594207
2,2024-12-05 01:00:00+00:00,4.2955,93.0,3.263425,1.556167,53.0,0.1,0.1,0.0,0.0,0.0,61.0,1025.000000,1020.217346,100.0,99.0,100.0,0.0,5520.0,0.0,0.000000,0.058281,8.049845
3,2024-12-05 02:00:00+00:00,4.2955,94.0,3.414988,1.446469,48.0,0.0,0.0,0.0,0.0,0.0,3.0,1025.500000,1020.714966,100.0,97.0,100.0,0.0,4180.0,0.0,0.000000,0.049955,9.000000
4,2024-12-05 03:00:00+00:00,4.2455,92.0,3.060890,1.087197,43.0,0.0,0.0,0.0,0.0,0.0,3.0,1024.800049,1020.017334,98.0,95.0,85.0,0.0,5820.0,0.0,0.000000,0.066374,10.661107
5,2024-12-05 04:00:00+00:00,4.1955,93.0,3.164258,1.282148,18.0,0.0,0.0,0.0,0.0,0.0,3.0,1024.300049,1019.518921,100.0,100.0,100.0,0.0,5200.0,0.0,0.000000,0.057874,9.114471
6,2024-12-05 05:00:00+00:00,4.1955,93.0,3.164258,1.508842,13.0,0.0,0.0,0.0,0.0,0.0,3.0,1024.500000,1019.717957,97.0,93.0,64.0,0.0,5660.0,0.0,0.000000,0.057874,7.559999
7,2024-12-05 06:00:00+00:00,3.9955,94.0,3.117119,1.132628,3.0,0.0,0.0,0.0,0.0,0.0,61.0,1024.199951,1019.415771,96.0,96.0,0.0,0.0,3940.0,0.0,0.000000,0.048915,8.707238
8,2024-12-05 07:00:00+00:00,3.7455,93.0,2.718001,0.964650,8.0,0.0,0.0,0.0,0.0,0.0,3.0,1024.099976,1019.312195,100.0,100.0,0.0,0.0,4440.0,0.0,0.000000,0.056074,7.636753
9,2024-12-05 08:00:00+00:00,3.6455,91.0,2.312934,0.696416,5.0,0.0,0.0,0.0,0.0,0.0,3.0,1024.800049,1020.007019,100.0,100.0,44.0,0.0,6980.0,0.0,0.013710,0.071590,8.287822
